In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForMaskedLM, AdamW
import pandas as pd

In [ ]:
data = pd.read_csv("/content/Grammer-dataset.csv", encoding='latin1')
data

,Sentence,Corrected Sentence
0,"Bitcoin is for $7,094 this morning, which Coin...","Bitcoin goes for $7,094 this morning, accordin..."
1,The effect of widespread dud targets two face ...,"1. The effect of ""widespread dud"" targets two ..."
2,tax on sales of stores for non residents are s...,Capital Gains tax on the sale of properties fo...
3,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
4,this is is the latest Maintenance release of S...,This is is the latest maintenance release of S...
...,...,...
494,Download the ChitramTV .apk file in https://ch...,Download the ChitramTV .apk file from https://...
495,Rates: $135-190 depending on volume (and retain).,Rates: $135-190 depending on volume and retainer.
496,When early results suggested more tally than e...,When early results suggested a closer tally th...
497,"From 22 July to 7 October, major engineering a...","From 22 July to 7 October, major engineering w..."


In [ ]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from torch.nn.utils.rnn import pad_sequence

class GrammarCorrectionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = str(self.data.iloc[idx]['Sentence'])
        corrected_sentence = str(self.data.iloc[idx]['Corrected Sentence'])

        # Tokenize the input sentence and create inputs for BERT
        inputs = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Tokenize the corrected sentence to be used as labels
        labels = self.tokenizer.encode(corrected_sentence, add_special_tokens=False)

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(labels)
        }

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    # Pad sequences dynamically within each batch
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }
# Adjust training parameters according to your requirements
train_dataset = GrammarCorrectionDataset(data, tokenizer, max_length=128)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=collate_fn)


In [ ]:
# Adjust training parameters according to your requirements
train_dataset = GrammarCorrectionDataset(data, tokenizer, max_length=128)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)


In [ ]:
optimizer = AdamW(model.parameters(), lr=8e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    # Pad sequences dynamically within each batch
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }


# Adjust training parameters according to your requirements
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=10, collate_fn=collate_fn)

# Inside the training loop
for epoch in range(10):  # Adjust number of epochs as needed
    print("Epoch:", epoch)
    for idx, batch in enumerate(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Pad or truncate inputs and labels to the same length within each batch
        max_len = max(input_ids.shape[1], labels.shape[1])
        input_ids = F.pad(input_ids, (0, max_len - input_ids.shape[1]), value=tokenizer.pad_token_id)
        attention_mask = F.pad(attention_mask, (0, max_len - attention_mask.shape[1]), value=0)
        labels = F.pad(labels, (0, max_len - labels.shape[1]), value=-100)  # Masking label for padding tokens

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        print("Loss:", loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


Epoch: 0
Loss: 0.41240671277046204
Loss: 0.5323372483253479
Loss: 0.5219654440879822
Loss: 0.3894290030002594
Loss: 0.3428899347782135
Loss: 0.3026939630508423
Loss: 0.3603331446647644
Loss: 0.43149325251579285
Loss: 0.2847060561180115
Loss: 0.4089720547199249
Loss: 0.2977714240550995
Loss: 0.29344019293785095
Loss: 0.45953986048698425
Loss: 0.30770421028137207
Loss: 0.27839651703834534
Loss: 0.22236669063568115
Loss: 0.3763289451599121
Loss: 0.40026456117630005
Loss: 0.31880179047584534
Loss: 0.3355255722999573
Loss: 0.37236377596855164
Loss: 0.46835416555404663
Loss: 0.3466191291809082
Loss: 0.4014880061149597
Loss: 0.5452749729156494
Loss: 0.4696085751056671
Loss: 0.3266502618789673
Loss: 0.2359192669391632
Loss: 0.3417704403400421
Loss: 0.427783340215683
Loss: 0.34836795926094055
Loss: 0.4660123288631439
Loss: 0.3855840563774109
Loss: 0.294149786233902
Loss: 0.2153806984424591
Loss: 0.634885847568512
Loss: 0.4251079559326172
Loss: 0.41414594650268555
Loss: 0.17564357817173004
Loss:

In [ ]:
torch.save(model, 'bert-finetuned')

In [ ]:
# load model
model = torch.load('bert-finetuned')
# check model summary
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
import torch

def get_corrected_sentence(input_string, model, tokenizer, max_length=128, device='cpu'):
    # Tokenize the input sentence
    inputs = tokenizer.encode_plus(
        input_string,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = outputs.logits.argmax(dim=-1).squeeze().tolist()

    # Decode the predictions to get the corrected sentence
    corrected_sentence = tokenizer.decode(predictions, skip_special_tokens=True)

    return corrected_sentence

# Example usage:
input_string = "So that be communicator."
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
corrected_sentence = get_corrected_sentence(input_string, model, tokenizer, device=device)
print("Corrected Sentence:", corrected_sentence)


Corrected Sentence: so that becomes the communicator.
